In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from geojson_rewind import rewind
from urllib.request import urlopen
import json
import numpy as np

In [10]:
df = pd.read_csv("../data_clean/ACLED_UK_clean.csv", index_col=0)
df = df[df["climate"] == "Y"]

In [7]:
with urlopen('https://opendata.arcgis.com/datasets/48b6b85bb7ea43699ee85f4ecd12fd36_4.geojson') as response:
    counties = json.load(response)

In [11]:
counties_corrected = rewind(counties,rfc7946=False)

In [12]:
step = 0.2
to_bin = lambda x: np.floor(x / step) * step
df["latBin"] = to_bin(df["latitude"])
df["lonBin"] = to_bin(df["longitude"])
groups = df.groupby(["latBin", "lonBin", "sub_event_type"])

In [30]:
df_map = groups.agg(lat=("latitude", "mean"), lon=("longitude", "mean"), count=("data_id", "size")).reset_index()

In [31]:
px.set_mapbox_access_token("pk.eyJ1IjoibHVpc2RpYmRpbiIsImEiOiJjbGE5bDgyM28wZXdrM3dsdmZqdGVucXN3In0.ZwFwkorjA9Pa7q1dKdHuIA")

fig = px.scatter_mapbox(df_map, lat="lat", lon="lon", labels={'label name':'label name'}, title='MAP TITLE', color="sub_event_type", size="count")

fig.update_layout(autosize=False,
    margin=dict(l=0, r=0, t=0, b=0),
    height=800,
    width=800)

fig.update_geos(fitbounds="locations", visible=False)

In [32]:
import plotly.offline as py

py.plot(fig, filename="protest_map.html")

'protest_map.html'